In [1]:
import pdb
from os import environ

import jax
import jax.numpy as jnp
import numpy as np
import diffrax
import sys
import argparse
import pymc_experimental as pmx
import arviz as az

sys.path.append("../models/")
from birtwistle_2007 import *

sys.path.append("../")
from sampling_utils import *

# tell jax to use 64bit floats
jax.config.update("jax_enable_x64", True)

/Users/natetest/miniforge3/envs/pymc/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = 'birtwistle_2007'
free_params  = 'Kmf52,koff57,EGF_off,kcat90,koff91,koff40,koff88,koff95,kon91,kcat94,koff42,kon93,kcat92,koff46,kcon49,koff41,Vmaxr52,koff44,kon89,kon95'
data_file = '../../../results/MAPK/HF_96_synthetic_data.csv' 
nsamples = 1
savedir  = '../../../results/MAPK/param_est/'
input_state  = 'E' 
EGF_conversion_factor = 1 
ERK_states = 'ERKstar'
prior_family = "[['LogNormal(mu=6.3015942750942955)',['sigma']],['LogNormal(mu=-0.7927465457827001)',['sigma']],['LogNormal(mu=-4.045554398052669)',['sigma']], ['LogNormal(mu=2.995917256443601)',['sigma']],['LogNormal(mu=4.60515318584359)',['sigma']],['LogNormal(mu=1.1330459209859989)',['sigma']],['LogNormal(mu=1.367493731656173)',['sigma']],['LogNormal(mu=4.6051931857235955)',['sigma']],['LogNormal(mu=-1.626584071269071)',['sigma']],['LogNormal(mu=-0.003405793134832821)',['sigma']],['LogNormal(mu=1.2583189340660492)',['sigma']],['LogNormal(mu=-1.6079390363103645)',['sigma']],['LogNormal(mu=-1.6074399097714274)',['sigma']],['LogNormal(mu=-0.655080979753489)',['sigma']],['LogNormal(mu=2.3004127351323884)',['sigma']],['LogNormal(mu=1.9528432026578095)',['sigma']],['LogNormal(mu=5.294697322086547)',['sigma']], ['LogNormal(mu=-0.8521429120504033)',['sigma']],['LogNormal(mu=-1.6109390385603677)',['sigma']],['LogNormal(mu=-1.6129440517633882)',['sigma']]]"
t1 = 10000

In [3]:
print('Processing model {}.'.format(model_name))

# try calling the model
try:
    model = eval(model_name + '(transient=False)')
except:
    print('Warning Model {} not found. Skipping this.'.format(model_name))

# get parameter names and initial conditions
p_dict, plist = model.get_nominal_params()
y0_dict, y0 = model.get_initial_conditions()

# load the data
inputs, data = load_data(data_file)

# convert EGF to required units
inputs_native_units = inputs * EGF_conversion_factor

# get the params to sample
analyze_params = free_params.split(',')
free_param_idxs = [list(p_dict.keys()).index(p) for p in analyze_params]

# get the EGF index and ERK indices
state_names = list(y0_dict.keys())
EGF_idx = state_names.index(input_state)
ERK_indices = [state_names.index(s) for s in ERK_states.split(',')]

# construct the strings to make priors and constants
prior_param_dict = set_prior_params(model_name, list(p_dict.keys()), plist, free_param_idxs,
                                    upper_mult=100, lower_mult=0.01,prior_family=prior_family,
                                    savedir='./')

# make initial conditions that reflect the inputs
y0_EGF_ins = construct_y0_EGF_inputs(inputs_native_units, np.array([y0]), EGF_idx)

# construct the pymc model
pymc_model_auto = build_pymc_model(prior_param_dict, data, y0_EGF_ins, 
                ERK_indices, t1, diffrax.ODETerm(model))

plt.close('all')

Processing model birtwistle_2007.
Vmaxr52 19927.73 1.992773
pm.LogNormal("Vmaxr52",sigma=2.349619800594078, mu=5.294697322086547, )
Kmf52 54544.079999999994 5.454408
pm.LogNormal("Kmf52",sigma=2.349619800594078, mu=6.3015942750942955, )
koff46 51.94 0.005194
pm.LogNormal("koff46",sigma=2.349619800594077, mu=-0.655080979753489, )
EGF_off 1.7500000000000002 0.00017500000000000003
pm.LogNormal("EGF_off",sigma=2.3496198005940774, mu=-4.045554398052669, )
koff57 45.26 0.0045260000000000005
pm.LogNormal("koff57",sigma=2.349619800594077, mu=-0.7927465457827001, )
koff40 310.51 0.031051000000000002
pm.LogNormal("koff40",sigma=2.349619800594077, mu=1.1330459209859989, )
koff41 704.87 0.07048700000000001
pm.LogNormal("koff41",sigma=2.349619800594078, mu=1.9528432026578095, )
koff42 351.95 0.035195
pm.LogNormal("koff42",sigma=2.349619800594078, mu=1.2583189340660492, )
koff44 42.65 0.004265
pm.LogNormal("koff44",sigma=2.349619800594077, mu=-0.8521429120504033, )
koff88 392.55 0.039255
pm.LogNorma

In [4]:
pymc_model_auto.to_graphviz()

In [5]:
pymc_model = build_pymc_model_ABC(prior_param_dict, data, y0_EGF_ins, 
                    ERK_indices, t1, diffrax.ODETerm(model))

In [6]:
pymc_model.to_graphviz()

In [19]:
#################################
# Create jax functions to solve # 
#################################
def sol_op_jax(*params, y0):
    pred = solve_ss(diffrax.ODEModel(model), y0, params, t1)
    return pred

# get the jitted versions
sol_op_jax_jitted = jax.jit(sol_op_jax)

############################################
# Create pytensor Op and register with jax # 
############################################
class StimRespOp(Op):
    def __init__(y0):
        super().__init__()
        self.y0 = y0

    def make_node(self, *inputs):
        # Convert our inputs to symbolic variables
        inputs = [pt.as_tensor_variable(inp) for inp in inputs]
        # Assume the output to always be a float64 matrix
        outputs = [pt.matrix()]
        return Apply(self, inputs, outputs)

    def perform(self, node, inputs, outputs):
        result = sol_op_jax_jitted(*inputs, self.y0)
        outputs[0][0] = np.asarray(result, dtype="float64")
    
    def grad(self, inputs, output_grads):
        raise NotImplementedError("PyTensor gradient of StimRespOp not implemented")


# construct Ops and register with jax_funcify
sol_op0 = StimRespOp(y0_EGF_ins[0])
sol_op1 = StimRespOp(y0_EGF_ins[1])
sol_op2 = StimRespOp(y0_EGF_ins[2])
sol_op3 = StimRespOp(y0_EGF_ins[3])
sol_op4 = StimRespOp(y0_EGF_ins[4])
sol_op5 = StimRespOp(y0_EGF_ins[5])
sol_op6 = StimRespOp(y0_EGF_ins[6])
sol_op7 = StimRespOp(y0_EGF_ins[7])
sol_op8 = StimRespOp(y0_EGF_ins[8])
sol_op9 = StimRespOp(y0_EGF_ins[9])

@jax_funcify.register(StimRespOp)
def sol_op_jax_funcify(op, **kwargs):
    return sol_op_jax

############################
# Construct the PyMC model # 
############################
model = pm.Model()
with model:
    # loop over free params and construct the priors
    priors = []
    for key, value in prior_param_dict.items():
        # create PyMC variables for each parameters in the model
        prior = eval(value)
        priors.append(prior)

    # predict dose response
    p0 = sol_op(*priors)
    p1 = sol_op(*priors)
    p2 = sol_op(*priors, y0_EGF_ins[2])
    p3 = sol_op(*priors, y0_EGF_ins[3])
    p4 = sol_op(*priors, y0_EGF_ins[4])
    p5 = sol_op(*priors, y0_EGF_ins[5])
    p6 = sol_op(*priors, y0_EGF_ins[6])
    p7 = sol_op(*priors, y0_EGF_ins[7])
    p8 = sol_op(*priors, y0_EGF_ins[8])
    p9 = sol_op(*priors, y0_EGF_ins[9])

    llike0 = pm.Normal("llike0", mu=p0, sigma=0.1, observed=data[0][0])
    llike1 = pm.Normal("llike1", mu=p1, sigma=0.1, observed=data[1][0])
    llike2 = pm.Normal("llike2", mu=p2, sigma=0.1, observed=data[2][0])
    llike3 = pm.Normal("llike3", mu=p3, sigma=0.1, observed=data[3][0])
    llike4 = pm.Normal("llike4", mu=p4, sigma=0.1, observed=data[4][0])
    llike5 = pm.Normal("llike5", mu=p5, sigma=0.1, observed=data[5][0])
    llike6 = pm.Normal("llike6", mu=p6, sigma=0.1, observed=data[6][0])
    llike7 = pm.Normal("llike7", mu=p7, sigma=0.1, observed=data[7][0])
    llike8 = pm.Normal("llike8", mu=p8, sigma=0.1, observed=data[8][0])
    llike9 = pm.Normal("llike9", mu=p9, sigma=0.1, observed=data[9][0])

ShapeError: Dimensionality of data and RV don't match. (actual 0 != expected 2)

In [22]:
p3 = sol_op(*priors, y0_EGF_ins[3])
p3.eval()

TypeError: sol_op_jax() missing 1 required keyword-only argument: 'y0'
Apply node that caused the error: StimRespOp(VmaxPY{223.8776}, KmPY{486.1398}, kdeg{0.0259}, kf47{24.6048}, Vmaxr47{590.5058}, Kmf47{698.6022}, Kmr47{483.8622}, kf48{16.833}, Kmf48{715.5688}, Kmr48{324.9294}, PTEN{693.5786}, kf49{44.3501}, kr49{552.6746}, Kmf49{343.2483}, Kmr49{753.1667}, Kmr49b{381.2208}, kr49b{640.8212}, kf51{3.6515}, Vmaxr51{16.737}, Kmf51{599.7076}, Kmrb51{988.4496}, kf52{0.7745}, Vmaxr52, Kmf52, Kmr52{675.2994}, kf54{0.0538}, Vmaxr54{588.2671}, Kmf54{457.9645}, Kmr54{336.183}, kf55{0.2256}, Vmaxr55{646.9003}, Kmf55{460.9446}, Kmr55{643.9247}, kf38{279.9929}, kf39{385.7428}, kf50{389.1061}, a98{0.0849}, b98{0.1833}, koff46, EGF_off, HRGoff_3{0.0009}, HRGoff_4{0.0973}, koff4{0.1717}, koff5{4.3985}, koff6{2.6619}, koff7{8.0557}, koff8{9.1034}, koff9{5.5425}, koff57, koff16{0.5737}, koff17{4.6874}, koff18{2.2768}, koff19{2.3361}, koff20{0.6761}, koff21{4.7291}, koff22{3.6962}, koff23{2.3619}, koff2